In [4]:
import pandas as pd
import ast
import json
import numpy as np
df1 = pd.read_csv('Dataset\movies_dataset.csv')
df2 = pd.read_csv ('Dataset\credits.csv')

C:\Users\Vero\AppData\Local\Temp\ipykernel_13820\4044590860.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('Dataset\movies_dataset.csv')


In [5]:
def corregir_formato_json(json_value):
    if json_value is None:
        return None
    if isinstance(json_value, float):
        json_value = str(json_value)
    try:
        return json.loads(json_value)
    except (json.JSONDecodeError, TypeError):
        return None


In [6]:
def desanidar_belongs_to_collection(df1):
    df1['belongs_to_collection'] = df1['belongs_to_collection'].apply(corregir_formato_json)
    df1 = pd.concat([df1.drop(['belongs_to_collection'], axis=1), df1['belongs_to_collection'].apply(pd.Series)], axis=1)
    return df1

In [7]:
df1['belongs_to_collection'] = df1['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else None)

In [8]:
df1[['id_collection', 'name_collection', 'poster_path_collection', 'backdrop_path_collection']] = df1['belongs_to_collection'].apply(pd.Series)[['id', 'name', 'poster_path', 'backdrop_path']]

In [9]:
df1['production_companies'] = df1['production_companies'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else None)

In [10]:
df1['production_companies'] = df1['production_companies'].apply(lambda x: ', '.join([d['name'] for d in x]) if x else None)

In [11]:
df1.production_companies

0                                  Pixar Animation Studios
1        TriStar Pictures, Teitler Film, Interscope Com...
2                             Warner Bros., Lancaster Gate
3                   Twentieth Century Fox Film Corporation
4               Sandollar Productions, Touchstone Pictures
                               ...                        
45461                                                 None
45462                                          Sine Olivia
45463                              American World Pictures
45464                                            Yermoliev
45465                                                 None
Name: production_companies, Length: 45466, dtype: object

In [12]:
df1['genres'] = df1['genres'].apply(ast.literal_eval)


In [13]:
df1['genres'] = df1['genres'].apply(lambda x: ', '.join([d['name'] for d in x]))

In [14]:
df1['production_countries'] = df1['production_countries'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [15]:
df1['production_countries'] = df1['production_countries'].apply(lambda x: ', '.join([d['iso_3166_1'] for d in x]) if isinstance(x, list) else None)

In [16]:
df2['cast'] = df2['cast'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])

In [17]:
df2['actors'] = df2['cast'].apply(lambda x: ', '.join([d['name'] for d in x[:3]]) if x else None)


In [18]:
df2['crew'] = df2['crew'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else None)

In [19]:
df2['director'] = df2['crew'].apply(lambda x: ', '.join([d['name'] for d in x if d.get('job') == 'Director']) if isinstance(x, list) else '')

In [20]:
df2['director']

0           John Lasseter
1            Joe Johnston
2           Howard Deutch
3         Forest Whitaker
4           Charles Shyer
               ...       
45471    Hamid Nematollah
45472            Lav Diaz
45473      Mark L. Lester
45474    Yakov Protazanov
45475       Daisy Asquith
Name: director, Length: 45476, dtype: object

In [21]:
df1['spoken_languages'] = df1['spoken_languages'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [22]:
df1['spoken_languages'] = df1['spoken_languages'].apply(lambda x: ', '.join([d['iso_639_1'] for d in x])if isinstance(x, list) else None)

In [23]:
columns_to_drop = ['belongs_to_collection','video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
df1 = df1.drop(columns=columns_to_drop)

Aqui llenamos los campos nulos de las columnas Revenue y Budget con 0

In [24]:
df1['revenue'] = df1['revenue'].fillna(0)
df1['budget'] = df1['budget'].fillna(0)

Convierto las columnas Revenue y Budget en tipo Float

In [25]:
df1['revenue'] = pd.to_numeric(df1['revenue'], errors='coerce')
df1['budget'] = pd.to_numeric(df1['budget'], errors='coerce')


In [26]:
invalid_rows = df1[df1['revenue'].isna() | df1['budget'].isna()]

In [27]:
print(invalid_rows)

       budget                                             genres          id  \
19730     NaN  Carousel Productions, Vision View Entertainmen...  1997-08-20   
29503     NaN  Aniplex, GoHands, BROSTA TV, Mardock Scramble ...  2012-09-29   
35587     NaN  Odyssey Media, Pulser Productions, Rogue State...  2014-01-01   

      original_language  overview             popularity production_companies  \
19730             104.0  Released                    NaN                 None   
29503              68.0  Released                    NaN                 None   
35587              82.0  Released  Beware Of Frost Bites                 None   

      production_countries release_date  revenue  ...  spoken_languages  \
19730                 None            1      0.0  ...              None   
29503                 None           12      0.0  ...              None   
35587                 None           22      0.0  ...              None   

      status tagline title vote_average  vote_count  

In [28]:
df1['budget'].fillna(0, inplace=True)

Eliminamos los campos nulos de la columna Release Date

In [29]:
df1 = df1.dropna(subset=['release_date'])

Cambio el formato de fecha

In [30]:
df1['release_date'] = pd.to_datetime(df1['release_date'],errors='coerce')

Creo la columna Release Year

In [31]:
df1['release_year'] = df1['release_date'].dt.year
df1['release_year'] = df1['release_year'].fillna(0).astype(int)


Creo la columna Return

In [32]:
df1['return'] = df1.apply(lambda row: row['revenue'] / row['budget'] if row['revenue'] != 0 and row['budget'] != 0 else 0, axis=1)
df1['return'] = df1['return'].round(2)

Creo los dataframes que usare en las funciones

In [33]:
df1[['id', 'original_language']].to_csv('idioma.csv', index=False)

In [34]:
df1[['id', 'title','runtime','release_year']].to_csv('duracion.csv', index=False)

In [35]:
df1[['id', 'name_collection','revenue']].to_csv('collection.csv', index=False)

In [36]:
df1[['id','production_countries']].to_csv('country.csv', index=False)

In [37]:
df1[['id','production_companies','revenue']].to_csv('production.csv', index=False)

In [40]:
df3 = pd.DataFrame({
    'id': df1['id'],
    'director': df2['director'],
    'actors': df2['actors'],
    'return': df1['return'],
    'budget' : df1['budget'],
    'revenue': df1['revenue'],
    'title': df1['title'],
    'release_date':df1['release_date']
})

df3.to_csv('cast.csv', index=False)

In [45]:
df_concated = pd.concat([df1, df2])

In [46]:
df_concated.to_csv('datos.csv', index=False)